# AI 스마트 분리수거 로봇 - YOLOv8 학습 노트북

**Google Colab (무료 T4 GPU) 에서 실행**

- 모델: YOLOv8 Nano (yolov8n) - 3.2M 파라미터
- 데이터: TOD + 직접 촬영 (6클래스, 5000장 목표)
- 목표: mAP50 85%+, 노트북 30+ FPS

## 1. 환경 설정

In [ ]:
# GPU 확인
!nvidia-smi

In [ ]:
# 라이브러리 설치
!pip install -q ultralytics roboflow

In [ ]:
import os
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt
import torch

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. 데이터셋 준비

### 방법 A: Roboflow에서 직접 다운로드 (권장)

In [ ]:
# 방법 A: Roboflow 데이터셋 다운로드
# Roboflow에서 프로젝트 생성 후 API 키와 프로젝트 정보 입력

from roboflow import Roboflow

# TODO: 자신의 Roboflow API 키로 교체
rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("YOUR_WORKSPACE").project("waste-sorting")
version = project.version(1)
dataset = version.download("yolov8")

DATA_YAML = dataset.location + "/data.yaml"
print(f"데이터셋 경로: {DATA_YAML}")

In [ ]:
# 방법 B: Google Drive에서 로드 (직접 준비한 데이터셋)
# from google.colab import drive
# drive.mount('/content/drive')
# DATA_YAML = '/content/drive/MyDrive/waste-sorter/data/dataset.yaml'

In [ ]:
# 방법 C: TOD GitHub에서 직접 다운로드
# !git clone https://github.com/jms0923/tod.git /content/tod
# 이후 라벨 재매핑 필요 (download_tod.py 참조)

## 3. 모델 학습

In [ ]:
# YOLOv8 Nano - COCO 사전학습 가중치에서 전이학습
model = YOLO('yolov8n.pt')

# 학습 실행
results = model.train(
    data=DATA_YAML,
    epochs=50,
    imgsz=640,
    batch=16,
    name='waste_sorter',
    patience=10,
    save=True,
    save_period=10,
    device=0,
    workers=2,
    # 데이터 증강
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10.0,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
)

## 4. 성능 평가

In [ ]:
# 학습된 최적 모델로 평가
best_model = YOLO('runs/detect/waste_sorter/weights/best.pt')
metrics = best_model.val(data=DATA_YAML)

print(f"\n{'='*40}")
print(f"mAP50:     {metrics.box.map50:.4f}")
print(f"mAP50-95:  {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall:    {metrics.box.mr:.4f}")
print(f"{'='*40}")

In [ ]:
# 학습 결과 시각화 (loss, mAP 곡선)
from IPython.display import Image, display

# 학습 그래프
results_img = Path('runs/detect/waste_sorter/results.png')
if results_img.exists():
    display(Image(filename=str(results_img), width=800))

# 혼동행렬
cm_img = Path('runs/detect/waste_sorter/confusion_matrix.png')
if cm_img.exists():
    display(Image(filename=str(cm_img), width=600))

In [ ]:
# 검증 이미지에서 예측 시각화
val_img = Path('runs/detect/waste_sorter/val_batch0_pred.jpg')
if val_img.exists():
    display(Image(filename=str(val_img), width=800))

## 5. 모델 내보내기 (ONNX)

In [ ]:
# ONNX 변환 (노트북/데스크탑 추론용)
best_model.export(format='onnx', imgsz=640, simplify=True)
print('ONNX 변환 완료!')

# TFLite 변환 (Raspberry Pi용 - 보고서 참고용)
# best_model.export(format='tflite', imgsz=640)

In [ ]:
# 추론 속도 벤치마크
import time
import numpy as np

test_img = np.random.randint(0, 255, (640, 640, 3), dtype=np.uint8)

# Warmup
for _ in range(5):
    best_model(test_img, verbose=False)

# 벤치마크
times = []
for _ in range(50):
    t0 = time.time()
    best_model(test_img, verbose=False)
    times.append(time.time() - t0)

avg_ms = np.mean(times) * 1000
fps = 1000 / avg_ms
print(f"평균 추론 시간: {avg_ms:.1f}ms")
print(f"FPS: {fps:.1f}")

## 6. 모델 다운로드

In [ ]:
# Google Drive에 저장
from google.colab import drive
drive.mount('/content/drive')

import shutil
save_dir = Path('/content/drive/MyDrive/waste-sorter-models')
save_dir.mkdir(parents=True, exist_ok=True)

# 모델 파일 복사
shutil.copy('runs/detect/waste_sorter/weights/best.pt', save_dir / 'best.pt')
shutil.copy('runs/detect/waste_sorter/weights/best.onnx', save_dir / 'best.onnx')

# 학습 결과 이미지 복사 (보고서용)
for img_name in ['results.png', 'confusion_matrix.png', 'val_batch0_pred.jpg']:
    src = Path(f'runs/detect/waste_sorter/{img_name}')
    if src.exists():
        shutil.copy(src, save_dir / img_name)

print(f'모델 저장 완료: {save_dir}')
!ls -la {save_dir}

In [ ]:
# 또는 직접 다운로드
from google.colab import files
files.download('runs/detect/waste_sorter/weights/best.pt')
# files.download('runs/detect/waste_sorter/weights/best.onnx')